In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pickle
import shap
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset = pd.read_csv('Your path files')

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
#卡方检验（比例模型）
from scipy.stats import chi2_contingency
from scipy.stats import chi2
data_a1 = dataset[dataset['Outcome'] == 0].iloc[:, 2:10]#####取outcome是0的二分类变量
data_b1 = dataset[dataset['Outcome'] == 1].iloc[:, 2:10]#####取outcome是1的二分类变量

#a = np.arange(9).reshape(3,3)
#定义数组
a = np.arange(data_a1.shape[1])
b = np.arange(data_a1.shape[1])
for i in range(0,data_a1.shape[1]):
    a[i] = data_a1.iloc[:,i].sum()
    b[i] = data_b1.iloc[:,i].sum()

chi_data = np.arange(6).reshape(3,2)

for i in range(0,data_a1.shape[1]):
    chi_data = np.array([[a[i],b[i],a[i]+b[i]],[data_a1.shape[0],data_b1.shape[0],data_a1.shape[0]+data_b1.shape[0]]])
    data = dataset.iloc[:, i+2]
    print(data.name)
    print(chi_data)
    stat, p, dof, expected = chi2_contingency(chi_data)
    
    #第一种print
    #print('自由度dof=%d' % dof)
    #print('期望分布')
    #print(expected)
    
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    #第二种print
    #print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    #if abs(stat) >= critical:
    #    print('拒绝原假设，两变量存在相关关系')
    #else:
    #    print('不能拒绝原假设，两变量相互独立')
    
    #第三种带有p值的print
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('拒绝原假设，两变量存在相关关系')
    else:
        print('不能拒绝原假设，两变量相互独立')

In [ ]:
#正态分布检验：W检验 Shapiro-Wilk检验
from scipy.stats import shapiro

for i in range(1,15):
    data = dataset.iloc[:, i+9]
    stat,p = shapiro(data)
    print(data.name)
    print("stat为：%f" %stat,"p值为：%f" %p)
    if p > 0.05:
        print('不能拒绝原假设，样本数据服从正态分布')
    else:
        print('不服从正态分布')

In [ ]:
#t检验，本章使用的是两独立样本t检验
from scipy.stats import ttest_ind
#from scipy.stats import ttest_rel 配合下面的检验类型使用对应函数包
#from scipy.stats import ttest_1samp
#from scipy.stats import f_oneway 多样本方差分析

data_a = dataset[dataset['Outcome'] == 0]
data_b = dataset[dataset['Outcome'] == 1]

for i in range(1,14):
    t = i+9
    data1 = data_a.iloc[:, t]
    data2 = data_b.iloc[:, t]
    stat, p = ttest_ind(data1, data2)
    #如果是配对关系样本 使用stat, p = ttest_rel(data1, data2)
    #单样本 t检验，只有一个data应该使用stats.ttest_1samp(data, 1)
    #多样本方差分析 stat, p = f_oneway(data1, data2, data3)
    print(data1.name,'的结果是')
    print('stat=%.3f, p=%.3f' % (stat, p))
    if p > 0.05:
        print('不能拒绝原假设，两样本集分布相同')
    else:
        print('拒绝原假设，样本集分布可能不同')



In [ ]:
#非参数检验，适用于非正态分布的数据，本文使用U检验
from scipy.stats import mannwhitneyu

data_a = dataset[dataset['Outcome'] == 0]
data_b = dataset[dataset['Outcome'] == 1]

for i in range(1,14):
    t = i+9
    data1 = data_a.iloc[:, t]
    data2 = data_b.iloc[:, t]
    stat, p = mannwhitneyu(data1, data2)
    
    print(data1.name,'的结果是')
    print('stat=%.3f, p=%.3f' % (stat, p))
    if p > 0.05:
        print('不能拒绝原假设，两样本集分布相同')
    else:
        print('拒绝原假设，样本集分布可能不同')


In [ ]:
#log归一化处理 结果差不多，一般不用，使用标准化特征预处理（naive bayes）结果能够有效提升
dataset.iloc[:,1:58]= dataset.iloc[:,1:58].apply(np.log10)
dataset.head()

In [ ]:
#如果没有使用log归一化，可以不用
print(dataset)

In [ ]:
rcParams['figure.figsize'] = 40, 80
plt.matshow(dataset.corr('kendall'), cmap=plt.cm.seismic)
#默认为pearson相关系数，完整代码为dataset.corr(method='pearson', min_periods=1),还有kendall：用于反映分类变量相关性的指标，即针对无序序列的相关系数，非正太分布的数据
#以及spearman：非线性的，非正太分析的数据的相关系数 后面的cmap是颜色，自己搜索颜色对应命令可以调节
plt.yticks(np.arange(dataset.shape[1]), dataset.columns)
plt.xticks(np.arange(dataset.shape[1]), dataset.columns, rotation=90)
plt.colorbar()

In [ ]:
dataset.hist()
a = dataset.corr()
a.to_csv('/Users/henryding/Library/CloudStorage/OneDrive-cmu.edu.cn/sci/复杂网络动态模型/supplementary/Clean Dataset coefficient(pearson).csv')

b = dataset.corr('kendall')
b.to_csv('/Users/henryding/Library/CloudStorage/OneDrive-cmu.edu.cn/sci/复杂网络动态模型/supplementary/Clean Dataset coefficient(kendall).csv')

c = dataset.corr('spearman')
c.to_csv('/Users/henryding/Library/CloudStorage/OneDrive-cmu.edu.cn/sci/复杂网络动态模型/supplementary/Clean Dataset coefficient(spearman).csv')

In [ ]:
dataset["Outcome"].describe()
dataset["Outcome"].unique()
print(dataset.corr()["Outcome"].abs().sort_values(ascending=False))

In [ ]:
dataset["Outcome"].describe()
dataset["Outcome"].unique()
print(dataset.corr('kendall')["Outcome"].abs().sort_values(ascending=False))

In [ ]:
dataset["Outcome"].describe()
dataset["Outcome"].unique()
print(dataset.corr('spearman')["Outcome"].abs().sort_values(ascending=False))

In [ ]:
y = dataset["Outcome"]

sns.countplot(y)

Target_temp = dataset.Outcome.value_counts()

print(Target_temp)

In [36]:
from sklearn.model_selection import train_test_split

predictors = dataset.drop("Outcome",axis=1)
Target = dataset["Outcome"]

X_train,X_test,Y_train,Y_test = train_test_split(predictors,Target,test_size=0.40,random_state=0)

In [ ]:
X_train.shape
X_test.shape
Y_train.shape
Y_test.shape

In [ ]:
#逻辑回归分类器

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(X_train,Y_train)

Y_pred_lr = lr.predict(X_test)
Y_pred_lr.shape
score_lr = round(accuracy_score(Y_pred_lr,Y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr)+" %")
# save the model to disk
filename = 'LogisticRegression.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
#logistic regression visualization
#为了不干扰后面继续使用dataset，进行一下copy
dataset_all = dataset
sns.pairplot(data=dataset_all,diag_kind='hist', hue= 'Outcome')
plt.show()
#plt.savefig('/Users/henryding/Desktop/plot.pdf')

In [ ]:
dataset_features = dataset_all.iloc[:,0:23]
print(dataset_features)

In [ ]:
for col in dataset_features.columns:
    sns.boxplot(x='Outcome', y=col, data=dataset_all)
    plt.title(col)
    fig = plt.gcf()#注意，下面三行一起用于导出plt结果为pdf
    plt.show()
    fig.savefig("/Users/henryding/Desktop/plot.pdf", format='pdf')

In [ ]:
#贝叶斯，高斯模型分类器（概率模型，很快）(高斯朴素贝叶斯分类模型)
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)

score_nb = round(accuracy_score(Y_pred_nb,Y_test)*100,2)

print("The accuracy score achieved using Naive Bayes is: "+str(score_nb)+" %")

In [ ]:
#标准化特征缩放（standardization feature scaling）
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline

# Code source: Tyler Lanigan <tylerlanigan@gmail.com>
#              Sebastian Raschka <mail@sebastianraschka.com>

# License: BSD 3 clause

RANDOM_STATE = 42
FIG_SIZE = (10, 7)


# Fit to data and predict using pipelined GNB and PCA
unscaled_clf = make_pipeline(PCA(n_components=2), GaussianNB())
unscaled_clf.fit(X_train, Y_train)
pred_test = unscaled_clf.predict(X_test)

# Fit to data and predict using pipelined scaling, GNB and PCA
std_clf = make_pipeline(StandardScaler(), PCA(n_components=2), GaussianNB())
std_clf.fit(X_train, Y_train)
pred_test_std = std_clf.predict(X_test)

# Show prediction accuracies in scaled and unscaled data.
print("\nPrediction accuracy for the normal test dataset with PCA")
print(f"{accuracy_score(Y_test, pred_test):.2%}\n")

print("\nPrediction accuracy for the standardized test dataset with PCA")
print(f"{accuracy_score(Y_test, pred_test_std):.2%}\n")

# Extract PCA from pipeline
pca = unscaled_clf.named_steps["pca"]
pca_std = std_clf.named_steps["pca"]

# Show first principal components
print(f"\nPC 1 without scaling:\n{pca.components_[0]}")
print(f"\nPC 1 with scaling:\n{pca_std.components_[0]}")

# Use PCA without and with scale on X_train data for visualization.
X_train_transformed = pca.transform(X_train)

scaler = std_clf.named_steps["standardscaler"]
scaled_X_train = scaler.transform(X_train)
X_train_std_transformed = pca_std.transform(scaled_X_train)

# visualize standardized vs. untouched dataset with PCA performed
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=FIG_SIZE)

target_classes = range(0, 2)
colors = ("blue", "red")
markers = ("^", "s")

for target_class, color, marker in zip(target_classes, colors, markers):
    ax1.scatter(
        x=X_train_transformed[Y_train == target_class, 0],
        y=X_train_transformed[Y_train == target_class, 1],
        color=color,
        label=f"class {target_class}",
        alpha=0.8,
        marker=marker,
    )

    ax2.scatter(
        x=X_train_std_transformed[Y_train == target_class, 0],
        y=X_train_std_transformed[Y_train == target_class, 1],
        color=color,
        label=f"class {target_class}",
        alpha=0.8,
        marker=marker,
    )

ax1.set_title("Training dataset after PCA")
ax2.set_title("Standardized training dataset after PCA")

for ax in (ax1, ax2):
    ax.set_xlabel("1st principal component")
    ax.set_ylabel("2nd principal component")
    ax.legend(loc="upper right")
    ax.grid()

plt.tight_layout()

plt.show()

In [ ]:
#SVM（支持向量机）分类器——大量运算
from sklearn import svm

sv = svm.SVC(kernel='linear')

sv.fit(X_train, Y_train)

Y_pred_svm = sv.predict(X_test)

Y_pred_svm.shape

score_svm = round(accuracy_score(Y_pred_svm,Y_test)*100,2)

print("The accuracy score achieved using Linear SVM is: "+str(score_svm)+" %")

In [ ]:
#KNN分类器
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train,Y_train)
Y_pred_knn=knn.predict(X_test)

Y_pred_knn.shape

score_knn = round(accuracy_score(Y_pred_knn,Y_test)*100,2)

print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")

In [ ]:
#决策树分类器
from sklearn.tree import DecisionTreeClassifier

max_accuracy = 0


for x in range(200):
    dt = DecisionTreeClassifier(random_state=x)
    dt.fit(X_train,Y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)


dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train,Y_train)
Y_pred_dt = dt.predict(X_test)

print(Y_pred_dt.shape)

score_dt = round(accuracy_score(Y_pred_dt,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")

In [ ]:
#随机森林分类器
from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0


for x in range(2000):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(X_train,Y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

rf = RandomForestClassifier(random_state=best_x)
rf.fit(X_train,Y_train)
Y_pred_rf = rf.predict(X_test)

Y_pred_rf.shape

score_rf = round(accuracy_score(Y_pred_rf,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")

In [ ]:
#神经网络（neural network models）
from sklearn.neural_network import MLPClassifier
nnm = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
nnm.fit(X_train,Y_train)
Y_pred_nnm = nnm.predict(X_test)

Y_pred_nnm.shape

score_nnm = round(accuracy_score(Y_pred_nnm,Y_test)*100,2)

print("The accuracy score achieved using Neural Network is: "+str(score_nnm)+" %")

In [ ]:
#XGboost分类器——类似stacking原理，合并了多个弱分类器的强分类器

import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
model = xgb_model.fit(X_train, Y_train)

Y_pred_xgb = xgb_model.predict(X_test)
Y_pred_xgb.shape

score_xgb = round(accuracy_score(Y_pred_xgb,Y_test)*100,2)

print("The accuracy score achieved using XGBoost is: "+str(score_xgb)+" %")

In [ ]:
#XGboost内容导出，获得权重模型
#SHAP处理

explainer = shap.Explainer(model)
shap_values = explainer(X_train)
shap.plots.waterfall(shap_values[0])#可视化
